In [1]:
import os, sys, math, pickle, time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from random import randint as ri
from random import uniform as ru

from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.losses import Huber
from keras.optimizers import Adam

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error

In [2]:
file = open("training_quad8_dataset_20100_20232112_125552.pkl", "rb")
training_data = pickle.load(file)
training_df = pd.DataFrame(training_data)

In [3]:
training_df

,sequence,step,increments,prev_j_positions,prev_base_pos_x,prev_base_pos_y,prev_base_pos_z,prev_base_ori_alpha,prev_base_ori_beta,prev_base_ori_gamma,post_j_positions,post_base_pos_x,post_base_pos_y,post_base_pos_z,post_base_ori_alpha,post_base_ori_beta,post_base_ori_gamma
0,0,0,"[0.11525049963219437, -0.09281151683004725, 0....","[-0.12015128135681152, 1.5527632236480713, -0....",0.118578,0.000000,0.000000,0.001776,-0.008311,0.001079,"[-0.0042269229888916016, 1.4716484546661377, 0...",0.004654,-0.022127,0.119621,0.000466,-0.000267,-0.033592
1,0,1,"[-0.02587766754917395, -0.12924746486957767, 0...","[-0.0042269229888916016, 1.4716484546661377, 0...",0.004654,-0.022127,0.119621,0.000466,-0.000267,-0.033592,"[-0.03027176856994629, 1.3417494297027588, 0.0...",0.008333,-0.013308,0.119512,-0.004121,-0.007747,-0.038558
2,0,2,"[0.024451154984391966, 0.0046065100464258955, ...","[-0.03027176856994629, 1.3417494297027588, 0.0...",0.008333,-0.013308,0.119512,-0.004121,-0.007747,-0.038558,"[-0.011545896530151367, 1.3305389881134033, 0....",0.005140,-0.009628,0.119403,-0.005793,-0.002855,-0.022720
3,0,3,"[-0.09621155427480091, 0.08237709238769023, -0...","[-0.011545896530151367, 1.3305389881134033, 0....",0.005140,-0.009628,0.119403,-0.005793,-0.002855,-0.022720,"[-0.11181020736694336, 1.3981926441192627, 0.0...",0.008121,-0.009540,0.119153,-0.005992,-0.002363,-0.005730
4,0,4,"[0.057529085407655306, 0.09925614321437072, -0...","[-0.11181020736694336, 1.3981926441192627, 0.0...",0.008121,-0.009540,0.119153,-0.005992,-0.002363,-0.005730,"[-0.061338186264038086, 1.4926025867462158, 0....",0.008182,-0.010234,0.119097,-0.004566,0.002879,-0.018702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,95,"[0.028381700251938133, -0.12149804662974291, 0...","[0.5118112564086914, 1.5494511127471924, 0.043...",0.041258,-0.066367,0.118506,-0.002534,0.026057,-0.275591,"[0.5410609245300293, 1.4306004047393799, 0.143...",0.032251,-0.062712,0.119638,-0.003584,0.004883,-0.260727
96,0,96,"[0.08753937301926161, -0.044504932936900145, 0...","[0.5410609245300293, 1.4306004047393799, 0.143...",0.032251,-0.062712,0.119638,-0.003584,0.004883,-0.260727,"[0.6225686073303223, 1.3609955310821533, 0.245...",0.026741,-0.057644,0.119556,-0.004848,0.006379,-0.275116
97,0,97,"[0.09372718541782811, -0.1145888336441924, -0....","[0.6225686073303223, 1.3609955310821533, 0.245...",0.026741,-0.057644,0.119556,-0.004848,0.006379,-0.275116,"[0.7135443687438965, 1.244004487991333, 0.1310...",0.015098,-0.050497,0.120418,-0.000247,0.007777,-0.277656
98,0,98,"[0.024508233751217558, 0.12836436307978683, -0...","[0.7135443687438965, 1.244004487991333, 0.1310...",0.015098,-0.050497,0.120418,-0.000247,0.007777,-0.277656,"[0.7465472221374512, 1.391512155532837, 0.0037...",0.022948,-0.053785,0.120354,-0.001880,0.006221,-0.292238


In [ ]:
#Function to split the training data into X,y datasets
def load_dataset():
    #file = open("training_quad8_dataset_23000_20232012_161313_.pkl", "rb")
    #file = open("training_quad8_dataset_23000_20232012_173325_.pkl", "rb")
    file = open("training_quad8_dataset_20100_20232112_125552.pkl", "rb")
    training_data = pickle.load(file)
    training_df = pd.DataFrame(training_data)

    #List to store the name for every joint data column
    increments_columns = []
    prev_j_positions_columns = []
    post_j_positions_columns = []


    #Creates the name for every column
    num_joints = len(training_data[-1]["increments"])
    for joint_n in range(num_joints):
        #List to split inputs per joint
        joint_inc_col_name = "increments_" + str(joint_n)
        increments_columns.append(joint_inc_col_name)

        prev_joint_pos_col_name = "prev_j_positions_" + str(joint_n)
        prev_j_positions_columns.append(prev_joint_pos_col_name)

        #List to split outputs per joint
        post_joint_pos_col_name = "post_j_positions_" + str(joint_n)
        post_j_positions_columns.append(post_joint_pos_col_name)


    #Input columns per joint
    increments_df = pd.DataFrame(training_df['increments'].to_list())
    increments_df.columns = increments_columns
    prev_j_positions_df = pd.DataFrame(training_df['prev_j_positions'].to_list())
    prev_j_positions_df.columns = prev_j_positions_columns

    X_df = pd.concat([increments_df, prev_j_positions_df, 
                    training_df["prev_base_pos_x"], training_df["prev_base_pos_y"], training_df["prev_base_pos_z"], 
                    training_df["prev_base_ori_alpha"], training_df["prev_base_ori_beta"], training_df["prev_base_ori_gamma"]], 
                    axis="columns")

    #Output columns per joint
    post_j_positions_df = pd.DataFrame(training_df['post_j_positions'].to_list())
    post_j_positions_df.columns = post_j_positions_columns

    y_df = pd.concat([post_j_positions_df, 
                    training_df["post_base_pos_x"], training_df["post_base_pos_y"], training_df["post_base_pos_z"], 
                    training_df["post_base_ori_alpha"], training_df["post_base_ori_beta"], training_df["post_base_ori_gamma"]], 
                    axis="columns")

    return(X_df,y_df)